In [1]:
import os
from dotenv import dotenv_values

In [2]:
config = dotenv_values(".env")

In [3]:
os.environ['FAISS_ENABLE_GPU']='ON' 
os.environ['HF_TOKEN'] = config.get("HF_TOKEN")

In [4]:
os.environ['FAISS_ENABLE_GPU']

'ON'

In [4]:

! pip install --no-binary :all: faiss-cpu

In [5]:
!pip install -U langchain-google-genai langchain langchain-community langchain-experimental  langchain-text-splitters -q --user

In [131]:
!pip install sentence_transformers langchain-huggingface --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum 1.18.0 requires transformers[sentencepiece]<4.40.0,>=4.26.0, but you have transformers 4.42.1 which is incompatible.
torchvision 0.17.2 requires torch==2.2.2, but you have torch 2.3.1 which is incompatible.


In [6]:
%pip install lark --user --quiet

Note: you may need to restart the kernel to use updated packages.


# Langchain

https://python.langchain.com/v0.1/docs/modules/chains/
### ConversationalRetrievalChain
https://api.python.langchain.com/en/latest/chains/langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain.html#langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain
### RetrievalQA
https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval_qa.base.RetrievalQA.html#langchain.chains.retrieval_qa.base.RetrievalQA

### Pdf loader
https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/
### Google Genai
https://python.langchain.com/v0.1/docs/integrations/chat/google_generative_ai/

# Vertexai
https://cloud.google.com/vertex-ai/docs/general/custom-service-account

# GeminiAI
https://ai.google.dev/gemini-api/docs/api-key

# Faiss
https://python.langchain.com/v0.2/docs/integrations/vectorstores/faiss/

https://github.com/facebookresearch/faiss/wiki

https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/


In [6]:
import os, tempfile
from langchain.prompts import PromptTemplate


from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain, RetrievalQA

from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from google.oauth2 import service_account
from dotenv import dotenv_values
import json
import vertexai
import chromadb.utils.embedding_functions as embedding_functions
import uuid
import faiss   
import itertools
import time
from langchain_huggingface import HuggingFaceEmbeddings

In [7]:
config = dotenv_values("./keys/.env")
with open("./keys/complete-tube-421007-9a7c35cd44e2.json") as source:
    info = json.load(source)

vertex_credentials = service_account.Credentials.from_service_account_info(info)
vertexai.init(
    project=config["PROJECT"],
    location=config["REGION"],
    credentials=vertex_credentials,
)
google_api_key = config["GEMINI-API-KEY"]
os.environ["GEMINI_API_KEY"] = google_api_key

In [8]:
ROOT= os.getcwd()
ROOT

'D:\\repos\\mlflow'

In [27]:
embeddings_retriever = GoogleGenerativeAIEmbeddings(
                        model="models/embedding-001",
                        credentials=vertex_credentials,
                        google_api_key=google_api_key,
                    )


#embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2" ,show_progress=True)

In [24]:
embeddings_retriever

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001F3A317B590>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=<google.oauth2.service_account.Credentials object at 0x000001F3F99DBE50>, client_options=None, transport=None, request_options=None)

In [11]:
mypath = "./docs"
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]

In [12]:
onlyfiles

['Baremo 2015.pdf',
 'BAREMO_AMA_BOE_RD_1971-1999.pdf',
 'BAREMO_PARA_LA_CALIFICACION_DEL_GRADO_DE_MINUSVALIA_Anexo_I_RD_1971-1999.pdf',
 'Clasificación de intervenciòn quirúrgica por nomenclator.pdf',
 'enfermedades profesionales de los agricultores.pdf',
 'Guia de Valoración Profesional.pdf',
 'GUIA_DE_VALORACION_DE_INCAPACIDAD_LABORAL_PARA_AP.pdf',
 'ley_enjuiciamiento_civil.pdf',
 'Proyecto de Ley Valoracion daÃ±os.pdf',
 'Tabla combinada de valoración de minusvalia.pdf',
 'Tablas_indemnizatorias_Baremo_2024.pdf',
 'Valoración del INSS de enfermedades para incapacidades.pdf']

In [146]:
def load_file(path):
    # load pdf file and transform into Langchain Documents
    loader = PyPDFLoader(path)
    pages = loader.load_and_split()
    return pages

def get_docs_to_add_vectorstore(pages, file, category="legal"):
    # get components to add to Faiis
    documents = []
    ids= []
    metadatas= []
   
    for page in pages:

        metadatas.append({"page": page.metadata.get("page"), "filename":file, "category":category})
        ids.append(uuid.uuid1().hex)
        documents.append(page.page_content)

        
    return  documents,  ids, metadatas 

In [121]:
uuid.uuid4().hex

'ca7efe0a09774e3d8502f5c161e72bc2'

In [186]:
path = os.path.join("docs", onlyfiles[0])
pages = load_file(path)

In [187]:
onlyfiles[0]

'Baremo 2015.pdf'

In [188]:
db2 = FAISS.from_documents(pages, embeddings_retriever)

In [189]:
pages[0]

Document(metadata={'source': 'docs\\Baremo 2015.pdf', 'page': 0}, page_content='BOLETÍN OFICIAL DEL ESTADO\nNúm. 228  Miércoles 23 de septiembre de 2015  Sec. I.   Pág. 84473\nI. DISPOSICIONES GENERALES\nJEFATURA DEL ESTADO\n10197 Ley 35/2015, de 22 de septiembre, de reforma del sistema para la valoración \nde los daños y perjuicios causados a las personas en accidentes de \ncirculación.\nFELIPE VI\nREY DE ESPAÑA\nA todos los que la presente vieren y entendieren.\nSabed: Que las Cortes Generales han aprobado y Yo vengo en sancionar la siguiente \nley:\nPREÁMBULO\nI\nHan transcurrido ya más de veinte años desde la entrada en vigor en 1995 del Sistema \nde valoración de daños y perjuicios causados a las personas en accidentes de circulación, \nal que en adelante nos referiremos con el nombre de «Baremo», que figura actualmente \ncomo Anexo en el Texto Refundido de la Ley sobre responsabilidad civil y seguro en la \ncirculación de vehículos a motor, aprobado por el Real Decreto Legislativ

In [190]:
len(pages)

673

In [192]:
print(db2.index.ntotal)

673


In [193]:
query = "que dice la Disposición final tercera. Título competencial?"
docs = db2.similarity_search(query)

In [207]:
docs[3].metadata

{'source': 'docs\\Baremo 2015.pdf', 'page': 34}

In [206]:
print(docs[3].page_content)

BOLETÍN OFICIAL DEL ESTADO
Núm. 228  Miércoles 23 de septiembre de 2015  Sec. I.   Pág. 84507
Subsección 1.ª Perjuicio personal básico (Disposiciones relativas a la tabla 3.A)
Artículo 136.  Determinación de la indemnización del perjuicio personal básico.
1. El perjuicio personal básico por lesión temporal es el perjuicio común que se 
padece desde la fecha del accidente hasta el final del proceso curativo o hasta la 
estabilización de la lesión y su conversión en secuela.
2. Su valoración económica se determina mediante la cantidad diaria 
establecida en la tabla 3.A.
Subsección 2.ª Perjuicio personal particular (Disposiciones relativas a la tabla 3.B)
Artículo 137.  Perjuicio personal por pérdida temporal de calidad de vida.
La indemnización por pérdida temporal de calidad de vida compensa el perjuicio 
moral particular que sufre la víctima por el impedimento o la limitación que las 
lesiones sufridas o su tratamiento producen en su autonomía o desarrollo personal.
Artículo 138.  Gra

In [208]:
len(docs)

4

In [209]:
db2.save_local("./faiss2", index_name="forensic")

In [212]:
db_load =FAISS.load_local(folder_path="./faiss2/", embeddings=embeddings_retriever, index_name="forensic", allow_dangerous_deserialization=True)

In [213]:
docs = db_load.similarity_search_with_relevance_scores(query)# filter={"source" :"docs\\Baremo 2015.pdf"})

In [214]:
len(docs)

4

In [215]:
docs[0][0].metadata

{'source': 'docs\\Baremo 2015.pdf', 'page': 8}

In [216]:
docs[0][0].page_content

'BOLETÍN OFICIAL DEL ESTADO\nNúm. 228  Miércoles 23 de septiembre de 2015  Sec. I.   Pág. 84481\nen su caso, el Consorcio de Compensación de Seguros, a una comparecencia en el \nplazo de cinco días, a fin de que pueda aportarse la oferta o la respuesta motivada, \no hacerse las alegaciones que consideren convenientes.\nSi en la comparecencia se produjera acuerdo entre las partes, el mismo será \nhomologado por el juez con los efectos de una transacción judicial.\nDe no alcanzarse el acuerdo, se dictará auto de cuantía máxima en el plazo de \ntres días desde la terminación de la comparecencia y contra el mismo no podrá \ninterponerse recurso alguno.»\nSeis. Se introduce un nuevo artículo 14, con el siguiente contenido:\n«Artículo 14.  Procedimiento de mediación en los casos de controversia.\n1. En caso de disconformidad con la oferta o la respuesta motivada y, en \ngeneral, en los casos de controversia, las partes podrán acudir al procedimiento de \nmediación de conformidad con lo previ

In [217]:
docs[0][1]

0.5645298260535585

In [147]:
# add documents to Faiss
documents_all =[]
metadatas_all = []
ids_all = []
category= "legal"
for file in onlyfiles:
    path = os.path.join("docs", file)
    pages = load_file(path)
    print(f" Loaded file {file} with {len(pages)} ")
    documents,  ids, metadatas = get_docs_to_add_vectorstore(pages, file, category)
    # add documents to collection
    documents_all = documents_all + documents
    metadatas_all = metadatas_all + metadatas
    ids_all = ids_all + ids

    print(f"number of documents: {len(documents_all)}")
    

 Loaded file Baremo 2015.pdf with 673 
number of documents: 673
 Loaded file BAREMO_AMA_BOE_RD_1971-1999.pdf with 139 
number of documents: 812
 Loaded file BAREMO_PARA_LA_CALIFICACION_DEL_GRADO_DE_MINUSVALIA_Anexo_I_RD_1971-1999.pdf with 283 
number of documents: 1095
 Loaded file Clasificación de intervenciòn quirúrgica por nomenclator.pdf with 254 
number of documents: 1349
 Loaded file enfermedades profesionales de los agricultores.pdf with 36 
number of documents: 1385
 Loaded file Guia de Valoración Profesional.pdf with 1211 
number of documents: 2596
 Loaded file GUIA_DE_VALORACION_DE_INCAPACIDAD_LABORAL_PARA_AP.pdf with 346 
number of documents: 2942
 Loaded file ley_enjuiciamiento_civil.pdf with 360 
number of documents: 3302
 Loaded file Proyecto de Ley Valoracion daÃ±os.pdf with 47 
number of documents: 3349
 Loaded file Tabla combinada de valoración de minusvalia.pdf with 5 
number of documents: 3354
 Loaded file Tablas_indemnizatorias_Baremo_2024.pdf with 42 
number of doc

In [148]:
metadatas_all[0]

{'page': 0, 'filename': 'Baremo 2015.pdf', 'category': 'legal'}

In [149]:
documents_all[0]

'BOLETÍN OFICIAL DEL ESTADO\nNúm. 228  Miércoles 23 de septiembre de 2015  Sec. I.   Pág. 84473\nI. DISPOSICIONES GENERALES\nJEFATURA DEL ESTADO\n10197 Ley 35/2015, de 22 de septiembre, de reforma del sistema para la valoración \nde los daños y perjuicios causados a las personas en accidentes de \ncirculación.\nFELIPE VI\nREY DE ESPAÑA\nA todos los que la presente vieren y entendieren.\nSabed: Que las Cortes Generales han aprobado y Yo vengo en sancionar la siguiente \nley:\nPREÁMBULO\nI\nHan transcurrido ya más de veinte años desde la entrada en vigor en 1995 del Sistema \nde valoración de daños y perjuicios causados a las personas en accidentes de circulación, \nal que en adelante nos referiremos con el nombre de «Baremo», que figura actualmente \ncomo Anexo en el Texto Refundido de la Ley sobre responsabilidad civil y seguro en la \ncirculación de vehículos a motor, aprobado por el Real Decreto Legislativo 8/2004, de 29 \nde octubre. Durante este tiempo, el conjunto de reformas que 

In [150]:
ids_all[0]

'6f4ac15d3bc011ef86f0f4b3013a69ec'

In [151]:
db = FAISS.from_texts(texts=documents_all, embedding=embeddings_retriever, metadatas=metadatas_all, ids=ids_all)

In [152]:
print(db.index.ntotal)

3807


In [153]:
db.save_local("./faiss", index_name="forensic")

In [218]:
db_load_2 =FAISS.load_local(folder_path="./faiss/", embeddings=embeddings_retriever, index_name="forensic", allow_dangerous_deserialization=True)

In [219]:
print(db_load_2.index.ntotal)

3807


In [220]:
query = "Disposición final tercera. Título competencial"

In [221]:
docs2 = db_load_2.similarity_search_with_relevance_scores(query)# filter={"filename" :"Baremo 2015.pdf"})

In [222]:
len(docs2)

4

In [223]:
docs2[0]

(Document(metadata={'page': 35, 'filename': 'ley_enjuiciamiento_civil.pdf', 'category': 'legal'}, page_content='Art. 60. Conflicto negativo de competencia territorial.– 1. Si la \ndecisión de inhibición de  un tribunal por falta de  competencia territorial \nse hubiere adoptado en virtud de declinatoria o con audiencia de todas \nlas partes, el tribunal al  que se remitieren las actuaciones estará a lo \ndecidido y no podrá declarar de oficio  su falta de compet encia territorial. \n2. Si la decisión de inhibición po r falta de competen cia territorial no \nse hubiese adoptado con audiencia de todas las partes, el tribunal a quien \nse remitieran las actuaciones podrá declarar de oficio su falta de competencia territorial cuando ésta deba determinarse en virtud de reglas imperativas. \n3. La resolución que declare la fa lta de competencia mandará remitir \ntodos los antecedentes al tribunal in mediato superior común, que decidirá \npor medio de auto, sin ulterior recu rso, el tribunal 

In [170]:
# get vectorstore from Presistem directory

In [240]:
retriever = db.as_retriever(
                        search_kwargs={"k": 5},
     filter={"filename" :"Baremo 2015.pdf"}
                    )

In [241]:
retriever.allowed_search_types

('similarity', 'similarity_score_threshold', 'mmr')

In [242]:
llm = ChatGoogleGenerativeAI(
                    model="gemini-1.5-pro-001", credentials=vertex_credentials
                )

In [252]:
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
# filter={"source" :"docs\\Baremo 2015.pdf"}
chain = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, return_source_documents=True)

In [253]:
chat_history = []

query = "Dame INDEMNIZACIONES POR CAUSA DE MUERTE PERJUICIO PERSONAL BÁSICO en la Categoría 1. El Cónyuge viudo,  segun el Boletin OFIcial del es del 2015?"
result = chain.invoke({"question": query, "chat_history": chat_history})

print(result['answer'])

Hasta 15 años de convivencia, si la victima tenía hasta 67 años 90.000 €
Hasta 15 años de convivencia, si la victima tenía desde 67 hasta 80 años 70.000 €
Hasta 15 años de convivencia, si la victima tenía más de 80 años 50.000 €
Por cada año adicional de convivencia o fracción con independencia de la edad de la victima. 1.000 € 



In [254]:
#print(result['source_documents'])

In [255]:
chat_history = [(query, result["answer"])]

In [256]:
query = "Y de la misma tabla Categoría 2. Los Ascendientes?"
result = chain.invoke({"question": query, "chat_history": chat_history})

print(result['answer'])

A cada progenitor, si el hijo fallecido tenía hasta 30 años 70.000 €
A cada progenitor, si el hijo fallecido tenía más de 30 años 40.000 €
A cada abuelo, sólo en caso de premoriencia del progenitor de su rama
familiar 20.000 € 



In [257]:
chat_history = [(query, result["answer"])]

In [258]:
query = "Y de la misma tabla puedes darme la Categoría 3. Los Descendientes?"
result = chain.invoke({"question": query, "chat_history": chat_history})

print(result['answer'])

Lo siento, pero la información proporcionada no contiene una tabla de descendientes ni ninguna referencia a categorías y descendientes. Por lo tanto, no puedo responder a tu pregunta. 



In [259]:
chat_history = [(query, result["answer"])]

In [260]:
query = "dame la Disposición final tercera. Título competencial"
result = chain.invoke({"question": query, "chat_history": chat_history})

print(result['answer'])

I'm sorry, but the provided text does not contain the information needed to answer your question about the third final provision and its competence title. 



In [261]:
retriever = db.as_retriever(
                        search_kwargs={"k": 5},
     filter={"filename" :"Baremo 2015.pdf"})

In [262]:
# Use RetrievalQA chain for orchestration
query = "Dame INDEMNIZACIONES POR CAUSA DE MUERTE PERJUICIO PERSONAL BÁSICO en la Categoría 1. El Cónyuge viudo,  segun el Boletin OFIcial del es del 2015?"
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever,  return_source_documents=True)
result = qa.invoke(query)


In [263]:
result.keys()

dict_keys(['query', 'result', 'source_documents'])

In [264]:
result['result']

'Hasta 15 años de convivencia, si la victima tenía hasta 67 años 90.000 €\nHasta 15 años de convivencia, si la victima tenía desde 67 hasta 80 años 70.000 €\nHasta 15 años de convivencia, si la victima tenía más de 80 años 50.000 €\nPor cada año adicional de convivencia o fracción con independencia de la edad de la victima. 1.000 € \n'

In [265]:
result['source_documents']

[Document(metadata={'page': 43, 'filename': 'Baremo 2015.pdf', 'category': 'legal'}, page_content='BOLETÍN OFICIAL DEL ESTADO\nNúm. 228  Miércoles 23 de septiembre de 2015  Sec. I.   Pág. 84516\nANEXO   \n \n    \n    \n    Categoría 1. El Cónyuge viudo\nHasta 15 años de convivencia, si la victima tenía hasta 67 años 90.000 €\nHasta 15 años de convivencia, si la victima tenía desde 67 hasta 80 años 70.000 €\nHasta 15 años de convivencia, si la victima tenía más de 80 años 50.000 €\nPor ca daañoadicionalde conv ivenciaofracción con independenciadela\nedad de la victima.1.000 €\nCategoría 2. Los Ascendientes\nA cada progenitor, si el hijo fallecido tenía hasta 30 años 70.000 €\nA cada progenitor, si el hijo fallecido tenía más de 30 años 40.000 €\nA cada abuelo, sólo en caso de premoriencia del progenitor de su rama\nfamiliar20.000 €\nCategoría 3. Los Descendientes\nA cada hijo que tenga hasta 14 años 90.000 €\nA cada hijo que tenga desde 14 hasta 20 años 80.000 €\nA cada hijo que tenga 